# Issue Commands to Change Settings

https://github.com/stilldavid/gopro-utils

In [1]:
import bs4

import data_io

import context

from gopro_helper import gopro
from gopro_helper import api
from gopro_helper import commands

# Camera Current Settings

In [27]:
k = gopro.GoProStatus(interval=1)

In [65]:
k.stop()

# Basic Commands

In [2]:
commands.set_mode_photo()

In [38]:
commands.set_mode_video()

In [53]:
commands.shutter_capture()

In [52]:
commands.shutter_stop()

In [16]:
%%time

urls = commands.get_data_urls()

CPU times: user 1.35 s, sys: 4 ms, total: 1.35 s
Wall time: 7.56 s


In [17]:
len(urls)

7992

---

In [2]:
from progress_bar import bar

In [18]:
def local_data(path_save='./data'):
    """Return list of locally-stored data files
    """
    files = data_io.find(path_save, ['*.JPG', '*.jpg', '*.MP4', '*.mp4'])
    names = [os.path.basename(f) for f in files]
    
    return names

In [19]:
def update_local_data(path_save='./data', delete=False):

    local_names = local_data(path_save)
    data_urls = commands.get_data_urls()
    
    for url in bar(data_urls):
        name = os.path.basename(url)
        if name not in local_names:
            f = commands.download(url, path_save)
                
        if delete:
            commands.delete_file(url)

In [20]:
update_local_data(delete=True)